In [ ]:
# importa bibliotecas necessárias
import pandas as pd
from mysql.connector import Error
from mysql.connector import errorcode
import mysql.connector

In [ ]:
# define componentes de autenticação
My_host = 'localhost'
My_db = 'bd_teste_fontes'
My_user = 'gd'
My_pw = 'Alpar@123'

In [ ]:
# define script de consulta
sql_tasks = (
"SELECT * "
"FROM tb_nx_tasks"
)

In [ ]:
# conecta no BD, executa a pesquisa
try:
    connection = mysql.connector.connect(host = My_host, database = My_db, user = My_user, password = My_pw)
    
    df_tasks = pd.read_sql(sql_tasks, con=connection)
    
except mysql.connector.Error as error:
    print("Failed to read record from MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        connection.close()
        print(f'{df_tasks.shape[0]} registros lidos em {df_tasks.shape[1]} colunas')
        print("MySQL connection is closed")

In [ ]:
# Altera os nomes das colunas
lst_novo_nome = ['Protocolo', 'Entidade', 'EntidadeCodigo', 'Servico', 'Usuario',
       'Grupo', 'DataHora_Conclusao', 'DataHora_Atendimento',
       'DataHora_Criacao', 'Acao', 'EncaminhadoPara', 'Comentario',
       'Apoio', 'ProcessoEncerrado', 'ProcessoCancelado',
       'MotivoCancelamento', 'Notificacao', 'StatusExterno',
       'Agendamento', 'DataHora_Agendamento', 'Categoria', 'GrupoResponsavel',
       'Prazo']
df_tasks.columns = lst_novo_nome

In [ ]:
# define as colunas que irão propagar dimensões e as transforma em tipo categoria
cat_lst = ['Entidade', 'Servico']
df_tasks[cat_lst] = df_tasks[cat_lst].apply(lambda x: x.astype('category'))

# cria os nomes das colunas que serão chave da categoria e cria as próprias 
ad_cat_lst = ['cat_' + x for x in cat_lst]
df_tasks[ad_cat_lst] = df_tasks[cat_lst].apply(lambda x: x.cat.codes)

In [ ]:
# faz as transformações necessárias
df_tasks = df_tasks.drop_duplicates().reset_index(drop=True).reset_index()
df_tasks.columns = ['PK_dim_Entidade', 'Entidade']

In [ ]:
df = pd.DataFrame({'col1':[1,2,3,4,5], 'col2':list('abcab'),  'col3':list('ababb')})
print(df.dtypes)

lst = ['col2', 'col3']
df[lst] = df[lst].apply(lambda x: x.astype('category'))

print(df.dtypes)

In [ ]:
df

In [ ]:
cat_columns = df.select_dtypes(['category']).columns
print(cat_columns)
ad_cat_columns = ['cat_' + x for x in cat_columns]
print(ad_cat_columns)

df[ad_cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df